In [1]:
import os

In [2]:
os.chdir("../")
%pwd

'd:\\codes\\mlops\\Kidney-Disease-Classification'

In [26]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class TrainingConfig:
    root_dir: Path
    trained_model_path : Path
    updated_base_model_path : Path
    training_path : Path
    params_epochs : int
    params_image_size : list
    params_batch_size : int
    params_is_augmentation : bool

In [27]:
from src.cnnClassifier.constants import *
from src.cnnClassifier.utils.utils import read_yaml , create_direcories

In [28]:
class ConfigurationManager:
    def __init__(self, CONFIG_FILE_PATH = CONFIG_FILE_PATH, PARAMS_FILE_PATH = PARAMS_FILE_PATH):
        self.config = read_yaml(CONFIG_FILE_PATH)
        self.params = read_yaml(PARAMS_FILE_PATH)
        create_direcories([self.config.artifacts_root])
    
    def get_training_config(self) -> TrainingConfig:
        config = self.config.training
        prepare_base_model = self.config.prepare_base_model
        params = self.params
        training_data = os.path.join(self.config.data_ingestion.unzip_dir , "kidney-ct-scan-image")
    
        create_direcories([config.root_dir])
        
        training_config = TrainingConfig(
            root_dir = Path(config.root_dir),
            trained_model_path = Path(config.trained_model_path),
            updated_base_model_path =Path(prepare_base_model.updated_base_model_path) ,
            training_path = Path(training_data),
            params_epochs = params.EPOCHS,
            params_image_size = params.IMAGE_SIZE,
            params_batch_size = params.BATCH_SIZE,
            params_is_augmentation = params.AUGMENTATION
        )
        return training_config

In [29]:
import sys
import urllib.request as request 
import tensorflow as tf
from src.cnnClassifier.exception.exception import customexception

In [30]:
class Training:
    def __init__(self, config : TrainingConfig):
        self.config = config
    
    def get_base_model(self):
        self.model = tf.keras.models.load_model(self.config.updated_base_model_path)
    
    def train_valid_generator(self):
        
        datagenerator_kwargs = dict(
            rescale = 1./255,
            validation_split = 0.2
        )
        
        dataflow_kwargs = dict(
            target_size = self.config.params_image_size[:-1],
            batch_size = self.config.params_batch_size,
            interpolation = 'bilinear'
        )
        
        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
          **datagenerator_kwargs   # type: ignore
        )
        
        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory = self.config.training_path,
            subset = 'validation',
            shuffle = True,
            **dataflow_kwargs # type: ignore
        )
        
        if self.config.params_is_augmentation:
            train_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
                rotation_range = 40,
                width_shift_range = 0.2,
                height_shift_range = 0.2,
                shear_range = 0.2,
                zoom_range = 0.2,
                horizontal_flip = True,
                **datagenerator_kwargs # type: ignore
            )
        else :
            train_datagenerator = valid_datagenerator
            
        self.train_generator = train_datagenerator.flow_from_directory(
                directory = self.config.training_path,
                subset = 'training',
                shuffle = True,
                **dataflow_kwargs # type: ignore
            )
    @staticmethod
    def save_model(model : tf.keras.Model, path : Path):
        model.save(path)
    
    
    def train(self):
        self.steps_per_epoch = self.train_generator.samples// self.train_generator.batch_size
        self.validation_steps = self.valid_generator.samples // self.valid_generator.batch_size

        self.model.fit(
            self.train_generator,
            epochs=self.config.params_epochs,
            steps_per_epoch=self.steps_per_epoch,
            validation_steps=self.validation_steps,
            validation_data=self.valid_generator
        )

        self.save_model(
            path=self.config.trained_model_path,
            model=self.model
        )

In [31]:
try :
    config = ConfigurationManager()
    training_config = config.get_training_config()
    training = Training(training_config)
    training.get_base_model()
    training.train_valid_generator()
    training.train()
except Exception as e :
    raise customexception(e, sys)

[2025-01-01 22:29:36,093: INFO: utils: yaml file : config\config.yaml read successfully]


[2025-01-01 22:29:36,117: INFO: utils: yaml file : params.yaml read successfully]
[2025-01-01 22:29:36,126: INFO: utils: directory : artifacts created successfully]
[2025-01-01 22:29:36,133: INFO: utils: directory : artifacts\training created successfully]
Found 93 images belonging to 2 classes.
Found 372 images belonging to 2 classes.
Epoch 1/20
37/37 [==============================] - 36s 860ms/step - loss: 0.5897 - accuracy: 0.7541 - val_loss: 0.7424 - val_accuracy: 0.5000
Epoch 2/20
37/37 [==============================] - 30s 809ms/step - loss: 0.4149 - accuracy: 0.8398 - val_loss: 0.7178 - val_accuracy: 0.5222
Epoch 3/20
37/37 [==============================] - 31s 834ms/step - loss: 0.3224 - accuracy: 0.8591 - val_loss: 0.6398 - val_accuracy: 0.5222
Epoch 4/20
37/37 [==============================] - 30s 795ms/step - loss: 0.2904 - accuracy: 0.8729 - val_loss: 0.6002 - val_accuracy: 0.7222
Epoch 5/20
37/37 [==============================] - 27s 733ms/step - loss: 0.2533 - accura